In [ ]:
import snowflake.snowpark as snp
import json
import getpass

# Initial Setup

We need a way to save state throughout the project.  We will initially login as the ACCOUNTADMIN role in order to setup some additional users as well as the compute resources we will need. 

We will specify a couple of different compute resources which allows us to scale up and down easily.  Most of the workflow can use an extra-small warehouse but for certain tasks (ie. feature engineering and model training) we may need larger compute.  By specifying them in the state dictionary we can easily select the correct compute for any particular task.
  
Update the <i>username, account_locator, cloud_region_id </i> in the state dictionary below with the initial user that was created with your trial account. 

You can copy and paste those from your Welcome to Snowflake email:
<i>Your account has been activated. Your username is <username> and your account URL is https://<account_locator>,.<cloud_region_id>.snowflakecomputing.com.

In [ ]:
state_dict = {
    "connection_parameters": {"user": "<username>",
                              "account": "<account_locator>.<cloud_region_id>",
                              "role": "ACCOUNTADMIN"
                             },
    "compute_parameters" : {"default_warehouse": "XSMALL_WH",  
                            "ds_warehouse": "LARGE_WH",  
                            },
    "additional_roles" : { "default_role": "PUBLIC",
                            "project_role": "ACCOUNTADMIN",  
                         }
}

In [ ]:
with open('./include/state.json', 'w') as sdf:
    json.dump(state_dict, sdf)

We will connect with username and password.  In a non-demo system it is very important to use properly secured passwords with secret managers and/or oauth.

In [ ]:
account_admin_pass = getpass.getpass('Enter pass for user with ACCOUNTADMIN access: ')
with open('./include/state.json') as sdf:
    state_dict = json.load(sdf)

state_dict['connection_parameters']['password'] = account_admin_pass

session = snp.Session.builder.configs(state_dict["connection_parameters"]).create()

Create a sample user which will be used for the hands-on-lab. Normally you will have different roles (and possibly different users) for data scientists, data engineers, ML engineers, etc.

In [ ]:
session.use_role('securityadmin')

demo_username='jack'

session.sql("CREATE USER IF NOT EXISTS "+demo_username+\
            " LOGIN_NAME = '"+demo_username+"'"+\
            " FIRST_NAME = 'SNOWPARK'"+\
            " LAST_NAME = 'SQUIRREL'"+\
            " EMAIL = 'jack@squirrelly.snowpark'"+\
            " DEFAULT_ROLE = 'PUBLIC'"+\
            " MUST_CHANGE_PASSWORD = FALSE")\
        .collect()

In [ ]:
session.use_role('securityadmin')
demo_user_password=getpass.getpass('Enter a new password for the demo user '+demo_username)
try:
    session.sql("ALTER USER "+demo_username+" SET PASSWORD = '"+demo_user_password+"'").collect()
except:
    pass

Create compute instance as per the state dictionary

In [ ]:
session.use_role('ACCOUNTADMIN')

session.sql("GRANT ROLE "+state_dict['additional_roles']['project_role']+' TO USER '+demo_username).collect()

for role in state_dict['additional_roles'].values():
    for wh in state_dict['compute_parameters'].values():
        session.sql("GRANT CREATE DATABASE ON ACCOUNT TO ROLE "+role).collect()
        session.sql("CREATE WAREHOUSE IF NOT EXISTS "+wh+\
                " WITH WAREHOUSE_SIZE = '"+wh.split('_')[0]+\
                "' WAREHOUSE_TYPE = 'STANDARD' AUTO_SUSPEND = 60 AUTO_RESUME = TRUE initially_suspended = true;")\
            .collect()
        session.sql("GRANT USAGE ON WAREHOUSE "+wh+" TO ROLE "+role).collect() 
        session.sql("GRANT OPERATE ON WAREHOUSE "+wh+" TO ROLE "+role).collect() 

Allow users to import data shares

In [ ]:
session.use_role('ACCOUNTADMIN')
session.sql("GRANT IMPORT SHARE ON ACCOUNT TO "+state_dict['additional_roles']['default_role']).collect()

Now update the state dictionary to use the non-admin account.


In [ ]:
state_dict['connection_parameters']['user'] = demo_username
state_dict['connection_parameters']['password'] = demo_user_password
state_dict['connection_parameters']['role'] = state_dict['additional_roles']['project_role']
state_dict['connection_parameters']['database'] = 'SQUIRRELLY_'+demo_username
state_dict['connection_parameters']['schema'] = 'DEMO'

Save the updated state dictionary for the project team to use

In [ ]:
with open('./include/state.json', 'w') as sdf:
    json.dump(state_dict, sdf)

Create a python function to simplify the users' steps of starting a session.



In [ ]:
%%writefile steps/snowpark_connection.py
def snowpark_connect(state_file='./include/state.json'):
    import snowflake.snowpark as snp
    import json
    
    with open(state_file) as sdf:
        state_dict = json.load(sdf)    
    
    session=None
    session = snp.Session.builder.configs(state_dict["connection_parameters"]).create()
    session.use_warehouse(state_dict['compute_parameters']['default_warehouse'])
    return session, state_dict

Test the function that users will use.



In [ ]:
from steps.snowpark_connection import snowpark_connect
session, state_dict = snowpark_connect()

In [ ]:
session.get_current_warehouse()

In [ ]:
session.close()
